In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import requests
import re
driver = webdriver.Chrome(r'C:\Users\nilsi\Desktop\Project\chromedriver') 
import selenium as SN
import pandas as pd

In [4]:
import os
os.getcwd()


'C:\\Users\\nilsi\\Desktop\\Project'

In [5]:
crawl_delay=12

def getPcode(page):
    pCodeList = []
    for i in range(1,page+1):
        print(i,"페이지 입니다")
        headers = {"Referer" : "http://prod.danawa.com/list/?cate=112758&15main_11_02",
               "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36"}

        params = {"page" : i, "listCategoryCode" : 758, "categoryCode" : 758, "physicsCate1":860, 
                  "physicsCate2":869,"physicsCate3":0, "physicsCate4":0, "viewMethod": "LIST", "sortMethod":"",
                  "listCount":30, "group": 11, "depth": 2, "brandName":"","makerName":"","searchOptionName":"",
                  "sDiscountProductRate":0, "sInitialPriceDisplay":"N", 
                  "sPowerLinkKeyword":"노트북", "oCurrentCategoryCode":"a:2:{i:1;i:96;i:2;i:758;}", 
                  "innerSearchKeyword":"",
                  "listPackageType":1, "categoryMappingCode":710, "priceUnit":0, "priceUnitValue":0, "priceUnitClass":"",
                  "cmRecommendSort":"N", "cmRecommendSortDefault":"N", "bundleImagePreview":"N", "nPackageLimit":5, 
                  "nPriceUnit":0,"bMakerDisplayYN":"Y","isDpgZoneUICategory": "N", "sProductListApi":"search"}

        res = requests.post("http://prod.danawa.com/list/ajax/getProductList.ajax.php", headers = headers, data=params)
        soup = BeautifulSoup(res.text, "html.parser")
        a = soup.findAll("a", attrs = {"name":"productName"})
        
        
        for i in range(len(a)):
            pCodeList.append(a[i]['href'][35:-12])
            
       

        time.sleep(crawl_delay-10)

        
        
    return pCodeList

pcode=getPcode(150)
#pcode=getPcode(5)

pcode

pcode=pd.DataFrame(pcode)
pcode.to_csv('Pcode.csv')

In [7]:
pcode=pd.read_csv('pcode.csv')
pcode=pcode['0']

In [8]:

def get_raw_laptop_table(pcode):
    driver = webdriver.Chrome('./chromedriver')
    driver.implicitly_wait(3) # 웹 자원이 로드되기까지의 대기 시간 지정

    driver.get('http://prod.danawa.com/info/?pcode='+str(pcode)+'&cate=112758#bookmark_product_information')
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    printTable = soup.find('div',{'class':'prod_spec'}) #스펙 표만 따오기
    printTable2 = soup.find('div',{'class':'lowest_area'}) #가격 표 따오기

    
    driver.close()
    
    printTable = re.sub('<.+?>','\n',str(printTable)).strip()
    printTable = [line.replace('\n','').replace('←','') for line in printTable.split('\n\n\n\n\n')]
    printTable = [line.split('  ') for line in printTable]
    printTable = [[j.strip() for j in i] for i in printTable][3::] 
    
    printTable2 = re.sub('<.+?>','\n',str(printTable2)).strip()
    printTable2 = [line.replace('\n','').replace('←','') for line in printTable2.split('\n\n\n\n\n')]
    printTable2 = [line.split('  ') for line in printTable2]
    printTable2 = [[j.strip() for j in i] for i in printTable2][3::]#기본적인 가공만 마침(공백과 같은 불량한 데이터가 섞여있다)

    return printTable, printTable2


In [9]:
#remove_empty 함수
def remove_empty(data):
    data = [item for sublist in data for item in sublist]
    for i in range(len(data)+50):
        try:
            if (data[i] =='' and data[i+1]=='' and data[i+2]==''):
                del data[i],data[i],data[i]
            elif (data[i] =='' and data[i+1]==''):
                del data[i],data[i]
            elif data[i] =='':
                del data[i]
                
        except IndexError : 
            continue
        
    data = [w.replace('(\t\t\t\t\t\t\t제조사 웹사이트 바로가기)\t\t\t\t\t\t\t\t\t\t\t\t', '') for w in data]
    data[1] = data[1].split(' ',1)
    data[1] = data[1][0] 
    
    # 메모리 특징과 HDD 특징이 붙어 있기 때문에 이를 분리해주는 코드 
    index = [i for i, s in enumerate(data) if 'HDD 특징' in s]  #HDD 특징이라는 스트링을 포함하는 원소의 인덱스를 추출
    data[index[0]:index[0]+1] = data[int(index[0])].split(' ',1) #쪼갠 결과를 줄줄이 붙이기
    
    index2 = [i for i, s in enumerate(data) if 'VRAM' in s]  #VRAM이라는 스트링을 포함하는 원소의 인덱스를 추출
    data[index2[0]:index2[0]+1] = data[int(index2[0])].split(' ',1) #쪼갠 결과를 줄줄이 붙이기, split함수의 1이라는 인자는 1개까지만 쪼개라는 의미
    
    index3 = [i for i, s in enumerate(data) if 'Kg' or 'g' in s]  #Kg이라는 스트링을 포함하는 원소의 인덱스를 추출
    data[index3[0]:index3[0]+1] = data[int(index3[0])].split(' ',1) #쪼갠 결과를 줄줄이 붙이기, split함수의 1이라는 인자는 1개까지만 쪼개라는 의미
    
    
    
    
    return data

In [10]:
def spec_to_dataframe(data):
    data.remove('디스플레이')
    data_index=data.index('디스플레이 특징')
    new_list=data[0:data_index]
    new_list[9:10]=new_list[9].split(' ',1)
    new_list.insert(10,'CPU클럭')
    
    data_index=data.index('메모리 용량')
    len_data=len(new_list)
    new_list[len_data:len_data+1]=(data[data_index:data_index+2])
    if new_list[-1] == '메모리 타입':
        new_list[-1] = '0'
    
    data_index=data.index('메모리 타입')
    len_data=len(new_list)
    new_list[len_data:len_data+1]=(data[data_index:data_index+2])
    if new_list[-1] == 'HDD 특징':
        new_list[-1] = '0'
    
    data_index=data.index('HDD 용량')
    len_data=len(new_list)
    new_list[len_data:len_data+1]=(data[data_index:data_index+2])
    if new_list[-1] == 'SSD 형태':
        new_list[-1] = '0'
    
    data_index=data.index('SSD 용량')
    len_data=len(new_list)
    new_list[len_data:len_data+1]=(data[data_index:data_index+2])
    if new_list[-1] =='스토리지 구성' :
        new_list[-1] = '0'
   
    data_index=data.index('GPU 종류')
    len_data=len(new_list)
    new_list[len_data:len_data+2]=(data[data_index+1:data_index+3])
    if new_list[-1] == '엔비디아':
        new_list[len_data+1]=(data[data_index+3])
        if new_list[-1] == 'AMD':
            new_list[len_data+1] =(data[data_index+4])

    data_index=data.index('VGA 메모리')
    len_data=len(new_list)
    new_list[len_data:len_data+1]=(data[data_index:data_index+2])
    if new_list[-1] =='GPU 특징' :
        new_list[-1] = '0'
        
    data_index=data.index('배터리')
    len_data=len(new_list)
    new_list[len_data:len_data+1]=(data[data_index:data_index+2])
    if new_list[-1] == '어댑터':
        new_list[-1] = '0'
        
    data_index=data.index('운영체제')
    len_data=len(new_list)
    new_list[len_data:len_data+1]=(data[data_index:data_index+2])
    if new_list[-1] == '제원':
        new_list[-1] = '없음'
        
    
    data_index=data.index('무게')
    len_data=len(new_list)
    new_list[len_data:len_data+1]=(data[data_index:data_index+2])
    if new_list[-1] == 'A/S 특징':
        new_list[-1] = '0'
    
    data_index=data.index('보증기간')
    len_data=len(new_list)
    new_list[len_data:len_data+1]=(data[data_index:data_index+2])
    if new_list[-1] == '용도':
        new_list[-1] = '0'
        
    try: data_index=data.index('두께')
    except ValueError:
        len_data=len(new_list)
        new_list[len_data:len_data+1]=['두께','0']
        
        
    len_data=len(new_list)
    new_list[len_data:len_data+1]=(data[data_index:data_index+2])
    if new_list[-1] == '무게':
        new_list[-1] = '0'
        
    list_col = new_list[0::2]
    list_ele = new_list[1::2]
    
    DF= pd.DataFrame(list(zip(list_col, list_ele)), 
               columns =['Name', 'Val']) 
        
    return DF.T

In [11]:
df=pd.DataFrame()
for p in range(2300,4000) :
    try :
        table = get_raw_laptop_table(pcode[p])
    
        price_1 = [item for sublist in table[1] for item in sublist]
        price_index=[i for i, s in enumerate(price_1) if '원' in s]
        price = price_1[price_index[0]]

        table = remove_empty(table[0])
        table = spec_to_dataframe(table)
        table.loc[:,21] = pd.Series(price,index=table.index)
        table[21]['Name']='가격'
        df=df.append(table.iloc[1,:])
        if p % 10 ==0:
            print(p)
        time.sleep(crawl_delay)
    except:
        pass




2300
2310
2320
2330
2340
2350
2360
2370
2380
2390
2400
2410
2420
2430
2440
2450
2460
2470
2480
2490
2500
2510
2520
2530
2540
2550
2560
2570
2580
2590
2600
2610
2620
2640
2650
2660
2670
2680
2690
2700
2710
2720
2730
2740
2750
2760
2770
2780
2790
2800
2810
2820
2830
2840
2850
2860
2870
2890
2900
2910
2920
2930
2940
2950
2960
2970
2980
2990
3000
3010
3020
3030
3040
3050
3060
3070
3080
3090
3110
3120
3130
3140
3150
3160
3170
3180
3190
3200
3210
3220
3230
3240
3250
3260
3270
3290
3300
3310
3320
3340
3350
3360
3370
3390
3400
3410
3420
3430
3440
3450
3460
3470
3480
3490
3500
3510
3520
3530
3540
3550
3570
3580
3590


Exception ignored in: 

In [12]:
df.to_csv('laptop_3.csv',encoding="CP949")

In [ ]:
df2=pd.DataFrame()
for p in range(1000,1300) :
    try :
        table = get_raw_laptop_table(pcode[p])
    
        price_1 = [item for sublist in table[1] for item in sublist]
        price_index=[i for i, s in enumerate(price_1) if '원' in s]
        price = price_1[price_index[0]]

        table = remove_empty(table[0])
        table = spec_to_dataframe(table)
        table.loc[:,21] = pd.Series(price,index=table.index)
        table[21]['Name']='가격'
        df2=df2.append(table.iloc[1,:])
        if p % 10 ==0:
            print(p)
        time.sleep(crawl_delay)
    except:
        pass

1000
1010
1020


In [12]:
df

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
Val,LG전자,인텔,코어i5-8세대,위스키레이크,i5-8265U,1.6GHz(3.9GHz),쿼드 코어,39.62cm(15.6인치),와이드 16:9,1920x1080(FHD),...,0,256GB,UHD 620,VRAM:시스템메모리공유,35Wh,운영체제 미포함,1.75Kg A/S 특징,1년,두께:19.9mm,"765,000원"
Val,LG전자,인텔,코어i5-8세대,위스키레이크,i5-8265U,1.6GHz(3.9GHz),쿼드 코어,39.62cm(15.6인치),와이드 16:9,1920x1080(FHD),...,0,256GB,UHD 620,VRAM:시스템메모리공유,72Wh,운영체제 미포함,1.09Kg A/S 특징,1년,두께:16.8mm,"1,268,000원"
Val,APPLE,인텔,코어i9-9세대,커피레이크-R,i9-9880H,2.3GHz(4.8GHz),옥타 코어,39.11cm(15.4인치),와이드 16:10,2880x1800,...,0,512GB,라데온 Pro 560X,VRAM:4GB,83.6Wh,Mac OS X,1.83Kg,1년,0,"2,764,000원"
Val,한성컴퓨터,인텔,코어i5-8세대,위스키레이크,i5-8265U,1.6GHz(3.9GHz),쿼드 코어,35.56cm(14인치),와이드 16:9,1920x1080(FHD),...,0,500GB,UHD 620,VRAM:시스템메모리공유,47Wh,운영체제 미포함,1.05Kg A/S 특징,1년,두께:16.8mm,"740,000원"


In [28]:
df.to_csv('laptop_1.csv',encoding="CP949")

table = get_raw_laptop_table(9293607)    
price_1 = [item for sublist in table[1] for item in sublist]
price_index=[i for i, s in enumerate(price_1) if '원' in s]
price = price_1[price_index[0]]


table = remove_empty(table[0])
table = spec_to_dataframe(table)
table.loc[:,21] = pd.Series(price,index=table.index)
table[21]['Name']='가격'
df=df.append(table.iloc[1,:])
print(p)
time.sleep(crawl_delay)
